<a href="https://colab.research.google.com/github/invinciblesohail/Tabular-Player-Series-October-2021-Challenge/blob/main/CatboostClassifier_for_TPSOct21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install catboost

     |████████████████████████████████| 76.4 MB 30 kB/s 


In [3]:
from numbers import Real
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import pipeline
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [5]:
! mkdir ~/.kaggle

In [8]:
! cp kaggle.json ~/.kaggle/

In [9]:
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
! kaggle competitions download tabular-playground-series-oct-2021

  0% 0.00/1.12M [00:00<?, ?B/s]
100% 1.12M/1.12M [00:00<00:00, 37.7MB/s]
 99% 433M/438M [00:14<00:00, 33.0MB/s]
100% 438M/438M [00:14<00:00, 32.1MB/s]
 99% 870M/877M [00:23<00:00, 58.9MB/s]
100% 877M/877M [00:23<00:00, 38.6MB/s]


In [11]:
!ls

kaggle.json  sample_submission.csv.zip	train.csv.zip
sample_data  test.csv.zip


In [12]:
! unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [13]:
! unzip test.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                


In [14]:
! unzip sample_submission.csv.zip

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   


In [22]:
!ls

kaggle.json  sample_submission.csv	test.csv      train.csv
sample_data  sample_submission.csv.zip	test.csv.zip  train.csv.zip


In [15]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file)
    df = reduce_mem_usage(df)
    return df

In [16]:
train = import_data('train.csv')
test = import_data('test.csv')

Memory usage of dataframe is 2189.64 MB
Memory usage after optimization is: 505.45 MB
Decreased by 76.9%
Memory usage of dataframe is 1091.00 MB
Memory usage after optimization is: 252.25 MB
Decreased by 76.9%


In [17]:
print("Null values in train data", train.isnull().sum().sum())
print("Null values in test data", test.isnull().sum().sum())

Null values in train data 0
Null values in test data 0


In [18]:
y = train['target']
train.pop('target')
train.pop('id')
test.pop('id')
X=train
del train

In [20]:
model = CatBoostClassifier(task_type='GPU',verbose=0)
kf = model_selection.StratifiedKFold(n_splits=5)
accuracies = []
for idx in kf.split(X, y):
    train_idx, test_idx = idx[0], idx[1]

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]

    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


    model.fit(X_train, y_train)

    preds = model.predict_proba(X_test)[:,1]
    fold_acc = metrics.roc_auc_score(y_test, preds)
    print("Fold_accurracy: ", fold_acc)
    accuracies.append(fold_acc)

print(-1.0*np.mean(accuracies)) 

Fold_accurracy:  0.8535828155360712
Fold_accurracy:  0.8559552853683281
Fold_accurracy:  0.8552411893964352
Fold_accurracy:  0.8515008424771426
Fold_accurracy:  0.8514759678537382
-0.8535512201263431
